In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import ensemble, metrics

In [2]:
# читаем массив данных
data = pd.read_csv('lenta_example.csv')
print(len(data))
data.head()

24467


,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
1,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина
2,https://lenta.ru/news/2018/12/15/ovechkin/,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт,Хоккей
3,https://lenta.ru/news/2018/12/15/newyear/,Названы регионы России с самым дорогим и дешев...,Производитель онлайн-касс «Эвотор» проанализир...,Экономика,Деньги
4,https://lenta.ru/news/2018/12/15/gaz/,Россию и Украину пригласили на переговоры по газу,Вице-президент Еврокомиссии Марош Шефчович при...,Экономика,Госэкономика


**Векторизация** - процесс превращения объекта в массив чисел

До:

$$ a = \text{"мама мыла раму"}, $$
$$ b = \text{"папа мыл апельсин"} $$

После:

|   | апельсин  |  мама | мыть  |  папа |  рама  |
|---|---|---|---|---|---|
|a |  0 | 1  | 1  |  0 | 1  |
| b |  1 | 0  | 1  |  1 | 0  |

In [3]:
# делаем векторизацию - превращаем текст в набор чисел
vectorizer=TfidfVectorizer(analyzer='word')
vectorizer.fit_transform(data['text'] )

<24467x195536 sparse matrix of type '<class 'numpy.float64'>'
	with 3151943 stored elements in Compressed Sparse Row format>

## Алгоритм действий
Дано: data + answers

- Делим данные: 

$$data \rightarrow train\_data + test\_data$$


- Обучаем: 

$$(train\_data + answers) \xrightarrow{\text{обучение}}  model.fit() \rightarrow \textit{обученная модель} $$


- Проверяем:

$$test\_data \xrightarrow{\text{предсказание}} model.predict()  \rightarrow \textit{ответы}$$


- Оцениваем результат:

$$Accuracy = \frac{Угаданные\ верно\ ответы}{Всего\ данных}$$

In [4]:
# формируем обучающую выборку
train_data = data.tail(10000)['text']
X_train = vectorizer.fit_transform(train_data)
y_train = data.tail(10000)['topic']

# формируем тестовую выборку
test_data = data.head(10000)['text']
X_test = vectorizer.transform(test_data)
y_test = data.head(10000)['topic']

In [5]:
y_train

14467          Экономика
14468           Ценности
14469              Спорт
14470           Культура
14471                Дом
14472          Экономика
14473    Наука и техника
14474              Спорт
14475                Дом
14476           Культура
14477           Ценности
14478                Дом
14479          Экономика
14480           Ценности
14481          Экономика
14482              Спорт
14483    Наука и техника
14484                Дом
14485                Дом
14486          Экономика
14487              Спорт
14488           Ценности
14489          Экономика
14490          Экономика
14491                Дом
14492    Наука и техника
14493              Спорт
14494                Дом
14495           Культура
14496          Экономика
              ...       
24437        Бывший СССР
24438          Экономика
24439              Спорт
24440        Бывший СССР
24441              Спорт
24442        Бывший СССР
24443              Спорт
24444        Бывший СССР
24445           Культура


In [6]:
# обучаем модель
model = LogisticRegression()
model.fit(X_train, y_train)
result = model.predict(X_test)

In [7]:
result

array(['Спорт', 'Бывший СССР', 'Спорт', ..., 'Экономика',
       'Наука и техника', 'Культура'], dtype=object)

In [8]:
# посмотрим, как распределены классы
data.groupby('topic')['url'].count().sort_values(ascending=False)

topic
Спорт              5458
Экономика          4915
Наука и техника    3511
Бывший СССР        3260
Культура           3171
Дом                2088
Ценности           2064
Name: url, dtype: int64

In [9]:
# посмотрим на результаты
print ('Доля правильных ответов:')
print ('train - ', model.score(X_train, y_train))
print ('test - ', model.score(X_test, y_test))

Доля правильных ответов:
train -  0.9784
test -  0.9297


In [10]:
pd.DataFrame(metrics.confusion_matrix(y_test, result), index=model.classes_, columns=model.classes_)

,Бывший СССР,Дом,Культура,Наука и техника,Спорт,Ценности,Экономика
Бывший СССР,1324,9,11,28,11,3,103
Дом,1,631,14,4,9,3,78
Культура,16,4,1229,14,14,3,27
Наука и техника,9,0,7,1309,4,3,55
Спорт,17,11,9,3,2167,0,9
Ценности,1,3,54,31,17,661,17
Экономика,23,24,6,24,10,14,1976


In [28]:
# попробуем проверить модель вручную
model.predict(vectorizer.transform(['Медведев']))

array(['Экономика'], dtype=object)

In [29]:
data[data['text'].str.contains('Медведев')]

,url,title,text,topic,tags
8,https://lenta.ru/news/2018/12/15/davos/,Россия передумала бойкотировать Давосский форум,Официальная делегация правительства России при...,Экономика,Госэкономика
130,https://lenta.ru/news/2018/12/13/blame/,Россияне назвали виновных в росте цен,Ответственность за проблемы в стране и рост ст...,Экономика,Госэкономика
148,https://lenta.ru/news/2018/12/13/benzin_in_neft/,Российские нефтяники заработали триллионы на ф...,Российские нефтяные компании смогли заработать...,Экономика,Госэкономика
223,https://lenta.ru/news/2018/12/12/mihelson/,Запад начал давить на Норвегию из-за помощи Ро...,Глава газовой компании НОВАТЭК и один из богат...,Экономика,Госэкономика
360,https://lenta.ru/news/2018/12/10/pensia/,Раскрыты размеры пенсий в России,В Пенсионном фонде России (ПФР) рассказали о р...,Экономика,undefined
393,https://lenta.ru/news/2018/12/10/no_cheap_alco...,Россиян оставили без доступного «Боярышника»,Премьер-министр Дмитрий Медведев запретил прод...,Экономика,Рынки
400,https://lenta.ru/news/2018/12/10/alina_zhenya/,Загитова и Медведева все выяснили,Российская фигуристка Алина Загитова рассказал...,Спорт,Зимние виды
415,https://lenta.ru/news/2018/12/10/rus_npz/,Российский рынок бензина столкнулся с новой пр...,Антипинский нефтеперерабатывающий завод — один...,Экономика,Рынки
423,https://lenta.ru/news/2018/12/10/slutskaya/,Фанаты Загитовой затравили еще одну олимпийску...,Фанаты олимпийской чемпионки по фигурному ката...,Спорт,Зимние виды
527,https://lenta.ru/news/2018/12/07/pens_no/,Отправленная ветераном труда прибавка к пенсии...,Руководитель регионального «Социального центра...,Экономика,Госэкономика


In [ ]:
# аренда жилья превысила спрос 2018 года
# на форуме инноваций Греф презентовал новую платформу для игры на бирже
# В Омске прошла выставка жареных гвоздей
# Трамп наложил новые тарифы на сталь из китая из-за этого было открыто несколько заводов

In [ ]:
model.predict(vectorizer.transform(['Названы регионы России с самым дорогим и дешевым']))

In [ ]:
model.predict(vectorizer.transform(['жареных']))